In [2]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from torchvision import transforms
from PIL import Image
import pandas as pd

In [54]:
# Load the CSV files
train_data_df = pd.read_csv('Released_Data/train_data.csv')
super_classes_df = pd.read_csv('Released_Data/superclass_mapping.csv')
sub_classes_df = pd.read_csv('Released_Data/subclass_mapping.csv')

# Rename
super_classes_df.rename(columns={'class': 'superclass_name'}, inplace=True)
sub_classes_df.rename(columns={'class': 'subclass_name'}, inplace=True)

# Merge the class names with the training data
train_data_df = train_data_df.merge(super_classes_df, left_on='superclass_index', right_on='index', how='left')
train_data_df = train_data_df.merge(sub_classes_df, left_on='subclass_index', right_on='index', how='left')

In [55]:
train_data_df.head()

,image,superclass_index,subclass_index,index_x,superclass_name,index_y,subclass_name
0,0.jpg,1,37,1,dog,37,"Maltese dog, Maltese terrier, Maltese"
1,1.jpg,0,42,0,bird,42,"oystercatcher, oyster catcher"
2,2.jpg,1,62,1,dog,62,"Afghan hound, Afghan"
3,3.jpg,1,31,1,dog,31,Shih-Tzu
4,4.jpg,0,4,0,bird,4,"great grey owl, great gray owl, Strix nebulosa"


In [56]:
train_data_df["superclass_name"].value_counts()

reptile    2388
dog        2084
bird       1850
Name: superclass_name, dtype: int64

In [57]:
# Create Dataset class for multilabel classification
class MultiClassImageDataset(Dataset):
    def __init__(self, ann_df, super_map_df, sub_map_df, img_dir, transform=None):
        self.ann_df = ann_df
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ann_df)

    def __getitem__(self, idx):
        img_name = self.ann_df['image'][idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        super_idx = self.ann_df['superclass_index'][idx]
        super_label = self.super_map_df['class'][super_idx]

        sub_idx = self.ann_df['subclass_index'][idx]
        sub_label = self.sub_map_df['class'][sub_idx]

        if self.transform:
            image = self.transform(image)

        return image, super_idx, super_label, sub_idx, sub_label

class MultiClassImageTestDataset(Dataset):
    def __init__(self, super_map_df, sub_map_df, img_dir, transform=None):
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self): # Count files in img_dir
        return len([fname for fname in os.listdir(self.img_dir)])

    def __getitem__(self, idx):
        img_name = str(idx) + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [61]:
train_ann_df = pd.read_csv('Released_Data/train_data.csv')
super_map_df = pd.read_csv('Released_Data/superclass_mapping.csv')
sub_map_df = pd.read_csv('Released_Data/subclass_mapping.csv')

train_img_dir = 'Released_Data/train_shuffle'
test_img_dir = 'Released_Data/test_shuffle'

image_preprocessing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0), std=(1)),
])

# Create train and val split
train_dataset = MultiClassImageDataset(train_ann_df, super_map_df, sub_map_df, train_img_dir, transform=image_preprocessing)

proportions = [.9, .1]
lengths = [int(p * len(train_dataset)) for p in proportions]
lengths[-1] = len(train_dataset) - sum(lengths[:-1])
#train_dataset, val_dataset = random_split(train_dataset, [0.9, 0.1])
# Since I'm using PyTorch 1.1.0, I can't use the above line of code
train_dataset, val_dataset = random_split(train_dataset, lengths)

# Create test dataset
test_dataset = MultiClassImageTestDataset(super_map_df, sub_map_df, test_img_dir, transform=image_preprocessing)

In [62]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.block1 = nn.Sequential(
                        nn.Conv2d(3, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.MaxPool2d(2, 2)
                      )

        self.block2 = nn.Sequential(
                        nn.Conv2d(32, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.MaxPool2d(2, 2)
                      )

        self.block3 = nn.Sequential(
                        nn.Conv2d(64, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.MaxPool2d(2, 2)
                      )

        self.fc1 = nn.Linear(4*4*128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3a = nn.Linear(128, 4)
        self.fc3b = nn.Linear(128, 88)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        super_out = self.fc3a(x)
        sub_out = self.fc3b(x)
        return super_out, sub_out

resnet18_model = torchvision.models.resnet18()



In [63]:
def SCL_fn(features, labels, tau=0.1, mask=None):
    """
    Supervised contrastive loss function.
    :param features: tensor of (batch size, feature vector dim)
    :param labels: tensor of (batch size)
    :param tau: temperature
    :return: loss
    """
    
    # Supervised contrastive loss (SCL) definition
    # https://openaccess.thecvf.com/content/CVPR2021/papers/Wang_Contrastive_Learning_Based_Hybrid_Networks_for_Long-Tailed_Image_Classification_CVPR_2021_paper.pdf
    # device = torch.device('cpu')
    batch_size = features.shape[0]
    labels_view = labels.contiguous().view(-1, 1)
    positive_pair_mask = torch.eq(labels_view, labels_view.T).float() # Used for identifying positive pairs with anchor
    self_contrast_mask = 1 - torch.eye(batch_size, batch_size) # Used for masking out self-contrast entries
    
    dot_products = (features @ features.T) / tau
    exp = torch.exp(dot_products)
    numerator_exp = exp * (positive_pair_mask * self_contrast_mask) # for each anchor, mask out self-contrast case and consider only same classes
    denominator_exp_sums = torch.sum(exp * self_contrast_mask, dim=1, keepdim=True) # for each anchor, mask out self-contrast case and consider all samples within that batch
    
    idx = numerator_exp != 0
    logs = torch.log(numerator_exp[idx] / denominator_exp_sums)
    anchor_sums = torch.sum(logs, dim=1)
    loss_per_anchor = (-1 / torch.sum(positive_pair_mask * self_contrast_mask, dim=1)) * anchor_sums
    loss = torch.sum(loss_per_anchor)
    

    
    # # device = torch.device('cpu')
    # # features = torch.unsqueeze(features, dim=1)

    # # if len(features.shape) < 3:
    # #     raise ValueError('`features` needs to be [bsz, n_views, ...],'
    # #                         'at least 3 dimensions are required')
    # # if len(features.shape) > 3:
    # #     features = features.view(features.shape[0], features.shape[1], -1)

    # # batch_size = features.shape[0]
    # # if labels is not None and mask is not None:
    # #     raise ValueError('Cannot define both `labels` and `mask`')
    # # elif labels is None and mask is None:
    # #     mask = torch.eye(batch_size, dtype=torch.float32).to(device)
    # # elif labels is not None:
    # #     labels = labels.contiguous().view(-1, 1)
    # #     if labels.shape[0] != batch_size:
    # #         raise ValueError('Num of labels does not match num of features')
    # #     mask = torch.eq(labels, labels.T).float().to(device)
    # # else:
    # #     mask = mask.float().to(device)

    # # contrast_count = features.shape[1]
    # # contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
    
    # # anchor_feature = contrast_feature
    # # anchor_count = contrast_count

    # # # compute logits
    # # anchor_dot_contrast = torch.div(
    # #     torch.matmul(anchor_feature, contrast_feature.T),
    # #     0.07)
    # # # for numerical stability
    # # logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
    # # logits = anchor_dot_contrast - logits_max.detach()

    # # # tile mask
    # # mask = mask.repeat(anchor_count, contrast_count)
    # # # mask-out self-contrast cases
    # # logits_mask = torch.scatter(
    # #     torch.ones_like(mask),
    # #     1,
    # #     torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
    # #     0
    # # )
    # mask = mask * logits_mask

    # # compute log_prob
    # exp_logits = torch.exp(logits) * logits_mask
    # log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

    # # compute mean of log-likelihood over positive
    # mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

    # # loss
    # loss = - (0.07 / 0.07) * mean_log_prob_pos
    # loss = loss.view(anchor_count, batch_size).mean()
    return loss

In [64]:
class SupConLoss(nn.Module):
    """Supervised Contrastive Learning: https://arxiv.org/pdf/2004.11362.pdf.
    It also supports the unsupervised contrastive loss in SimCLR"""
    def __init__(self, temperature=0.07, contrast_mode='all',
                 base_temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature

    def forward(self, features, labels=None, mask=None):
        """Compute loss for model. If both `labels` and `mask` are None,
        it degenerates to SimCLR unsupervised loss:
        https://arxiv.org/pdf/2002.05709.pdf

        Args:
            features: hidden vector of shape [bsz, n_views, ...].
            labels: ground truth of shape [bsz].
            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j
                has the same class as sample i. Can be asymmetric.
        Returns:
            A loss scalar.
        """
        # device = (torch.device('cuda')
        #           if features.is_cuda
        #           else torch.device('cpu'))
        device = torch.device("cpu")

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)

        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            self.temperature)
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)
        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
            0
        )
        mask = mask * logits_mask

        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # loss
        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos
        loss = loss.view(anchor_count, batch_size).mean()

        return loss

In [17]:
class MultiTailModel(nn.Module):
    # def __init__(self, model_name, target='superclass'):
    #     super().__init__()
    #     if model_name == 'resnet18':
    #         self.resnet_model = torchvision.models.resnet18(pretrained=True)
    #         # Replace classification head
    #         self.resnet_model.fc = nn.Linear(512, 2000)
    #         # non-linear MLP with one hidden layer
    #         self.projection_head = nn.Sequential(
    #             nn.Linear(2048, 512),
    #             nn.ReLU(),
    #             nn.Linear(512, 2000),
    #             nn.ReLU(),
    #         )
    #         if target == 'superclass':
    #             self.fc = nn.Linear(2000, 4)
    #         elif target == 'subclass':
    #             self.fc = nn.Linear(2000, 88)
    #         else:
    #             raise ValueError('target must be superclass or subclass')
            
    
    # def forward(self, x):
    #     backbone_output = self.resnet_model(x)
    #     backbone_output = F.relu(backbone_output)
    #     z = self.projection_head(backbone_output)
    #     z_normalized = F.normalize(z, p=2, dim=1)
    #     class_logits = self.fc(backbone_output)
    #     return z_normalized, class_logits
    def __init__(self, model_name, target='superclass'):
        super().__init__()
        if model_name == 'resnet18':
            self.encoder_network = torchvision.models.resnet18(pretrained=True)
            # Replace classification head
            self.encoder_network.fc = nn.Linear(512, 2048)
            # non-linear MLP with one hidden layer
            self.projection_head = nn.Sequential(
                nn.Linear(2048, 2048),
                nn.ReLU(),
                nn.Linear(2048, 128),
            )
            if target == 'superclass':
                self.fc = nn.Linear(2048, 4)
            elif target == 'subclass':
                self.fc = nn.Linear(2048, 88)
            else:
                raise ValueError('target must be superclass or subclass')
            
    
    def forward(self, x):
        encoder_output = self.encoder_network(x)
        # encoder_output = F.relu(encoder_output)
        encoder_output = F.normalize(encoder_output, p=2, dim=1)
        z = self.projection_head(encoder_output)
        z_normalized = F.normalize(z, p=2, dim=1)
        class_logits = self.fc(encoder_output)
        return z_normalized, class_logits
            
             

In [60]:
class Trainer():
    def __init__(self, model, criterion1, criterion2, optimizer, train_loader, val_loader, test_loader=None, device='cpu'):
        self.model = model
        self.contrast_criterion = criterion1
        self.class_criterion = criterion2
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def train_epoch(self):
        self.model.train()
        running_loss = 0.0
        running_contrastive_loss = 0.0
        running_classification_loss = 0.0
        for i, data in enumerate(self.train_loader):
            inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)
            self.optimizer.zero_grad()
            super_class_feature_vect, super_class_logits = self.model(inputs)
            # super_outputs, class_logits = self.model(torch.unsqueeze(inputs))       
            # loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
            contrastive_loss = self.contrast_criterion(super_class_feature_vect, super_labels)
            classification_loss = self.class_criterion(super_class_logits, super_labels)
            loss = contrastive_loss + classification_loss
            # print(loss)
            # print(loss.size())
            loss.backward()
            self.optimizer.step()

            print(f"Batch: {i}, Total Loss = {loss.item()} Contrastive Loss: {contrastive_loss.item()}, Classification Loss: {classification_loss.item()}")
            running_loss += loss.item()
            running_contrastive_loss += contrastive_loss.item()
            running_classification_loss += classification_loss.item()

        print(f'Training loss: {running_loss/i:.3f}')
        print(f'Training contrastive loss: {running_contrastive_loss/i:.3f}')
        print(f'Training classification loss: {running_classification_loss/i:.3f}')

    def validate_epoch(self):
        super_correct = 0
        sub_correct = 0
        total = 0
        running_loss = 0.0
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

                _, super_outputs = self.model(inputs)
                # loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
                # loss = self.criterion(super_outputs, super_labels)
                loss = self.contrast_criterion(super_outputs, super_labels)
                _, super_predicted = torch.max(super_outputs.data, 1)
                # _, sub_predicted = torch.max(sub_outputs.data, 1)

                total += super_labels.size(0)
                super_correct += (super_predicted == super_labels).sum().item()
                # sub_correct += (sub_predicted == sub_labels).sum().item()
                running_loss += loss.item()

        print(f'Validation loss: {running_loss/i:.3f}')
        print(f'Validation superclass acc: {100 * super_correct / total:.2f} %')
        # print(f'Validation subclass acc: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, return_predictions=False):
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        # Evaluate on test set, in this simple demo no special care is taken for novel/unseen classes
        test_predictions = {'image': [], 'superclass_index': [], 'subclass_index': []}
        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(self.device), data[1]

                super_outputs, sub_outputs = self.model(inputs)
                _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                test_predictions['image'].append(img_name[0])
                test_predictions['superclass_index'].append(super_predicted.item())
                test_predictions['subclass_index'].append(sub_predicted.item())

        test_predictions = pd.DataFrame(data=test_predictions)

        if save_to_csv:
            test_predictions.to_csv('example_test_predictions.csv', index=False)

        if return_predictions:
            return test_predictions

In [37]:
features = torch.randn(11, 5, requires_grad=True)
features = F.normalize(features, p=2, dim=1)
labels = torch.Tensor([0,0,0,1,1,0,0,0,2,1,2])
loss = SCL_fn(features, labels, tau=0.5)
print(loss)

tensor(530.1482, grad_fn=<SumBackward0>)


In [38]:
# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False)

# Init model and trainer
device = 'cpu'
model = MultiTailModel(model_name="resnet18", target="superclass").to(device)#CNN().to(device)
criterion1 = SCL_fn #nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
optimizer = optim.Adam(model.parameters(), lr=0.5)
trainer = Trainer(model, criterion1, criterion2, optimizer, train_loader, val_loader, test_loader)

In [39]:
# Training loop
for epoch in range(10):
    print(f'Epoch {epoch+1}')
    trainer.train_epoch()
    trainer.validate_epoch()
    print('')

print('Finished Training')

Epoch 1
Batch: 0, Total Loss = 18930.154296875 Contrastive Loss: 18928.76171875, Classification Loss: 1.3921678066253662
Batch: 1, Total Loss = 19407.208984375 Contrastive Loss: 19404.9375, Classification Loss: 2.2724287509918213
Batch: 2, Total Loss = 18685.212890625 Contrastive Loss: 18675.30078125, Classification Loss: 9.912602424621582
Batch: 3, Total Loss = 16993.552734375 Contrastive Loss: 16976.09375, Classification Loss: 17.458293914794922
Batch: 4, Total Loss = 17937.759765625 Contrastive Loss: 17924.634765625, Classification Loss: 13.124505996704102
Batch: 5, Total Loss = 16954.595703125 Contrastive Loss: 16944.89453125, Classification Loss: 9.701126098632812
Batch: 6, Total Loss = 17768.072265625 Contrastive Loss: 17765.205078125, Classification Loss: 2.866980791091919
Batch: 7, Total Loss = 17022.15625 Contrastive Loss: 17014.498046875, Classification Loss: 7.657649040222168
Batch: 8, Total Loss = 17531.15234375 Contrastive Loss: 17521.89453125, Classification Loss: 9.25836

KeyboardInterrupt: 

In [42]:
trainer.validate_epoch()

Validation loss: nan
Validation superclass acc: 85.94 %


In [33]:
# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False)

# Init model and trainer
device = 'cpu'
model = MultiTailModel(model_name="resnet18", target="superclass").to(device)#CNN().to(device)
criterion1 = SCL_fn #nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
optimizer = optim.SGD(model.parameters(), lr=0.5, momentum=0.9)
trainer = Trainer(model, criterion1, criterion2, optimizer, train_loader, val_loader, test_loader)

In [31]:
# Training loop
for epoch in range(10):
    print(f'Epoch {epoch+1}')
    trainer.train_epoch()
    trainer.validate_epoch()
    print('')

print('Finished Training')

Epoch 1


AttributeError: 'DataLoader' object has no attribute 'zero_grad'

In [65]:
torchvision.models.resnet18(pretrained=True)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [49]:
torchvision.models.resnet50(pretrained=True)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [52]:
torchvision.models.inception_v3(pretrained=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\Vedant Gannu/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [3]:
torchvision.models.densenet121(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\Vedant Gannu/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu